In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("/data/raw_stock_ind.csv")

In [22]:
import json
df["info_json"] = df["info_json"].str.replace("\'", "\"")
df["info_json"] = df["info_json"].str.replace("None", "\"None\"")

In [27]:
df = df.assign(company_symbol=lambda x: (json.loads(x["info_json"]["Symbol"]))  

SyntaxError: unexpected EOF while parsing (<ipython-input-27-42d61d37b45d>, line 1)

In [25]:
df

,Tax Effect Of Unusual Items,Tax Rate For Calcs,Normalized EBITDA,Total Unusual Items,Total Unusual Items Excluding Goodwill,Net Income From Continuing Operation Net Minority Interest,Reconciled Depreciation,Reconciled Cost Of Revenue,EBIT,Net Interest Income,...,Impairment Of Capital Assets,Selling And Marketing Expense,Other Income Expense,Rent Expense Supplemental,Rent And Landing Fees,Research And Development,Total Operating Income As Reported,Capital Gains,Gain On Sale Of Security,company_symbol
0,0.000000e+00,0.250000,1.767100e+10,NaN,NaN,1.151500e+10,1.847000e+09,5.598700e+10,1.582400e+10,-460000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,abcd
1,0.000000e+00,0.228569,1.795100e+10,1.667000e+09,1.667000e+09,1.113700e+10,3.073000e+09,5.580700e+10,1.487800e+10,-436000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,abcd
2,-1.182283e+08,0.236457,1.576600e+10,-5.000000e+08,-5.000000e+08,1.000500e+10,1.781000e+09,5.472200e+10,1.348500e+10,-379000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,abcd
3,0.000000e+00,0.245756,1.051000e+10,NaN,NaN,6.793000e+09,1.308000e+09,3.084900e+10,9.202000e+09,-189000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,abcd
4,0.000000e+00,0.254524,9.749000e+09,NaN,NaN,6.335000e+09,1.065000e+09,2.872300e+10,8.684000e+09,-174000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,abcd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1107,0.000000e+00,0.278915,5.181872e+09,0.000000e+00,0.000000e+00,3.214777e+09,7.226630e+08,9.612063e+09,4.459209e+09,-961000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,abcd
1108,0.000000e+00,0.289715,3.919495e+09,0.000000e+00,0.000000e+00,2.291491e+09,6.923470e+08,7.663952e+09,3.227148e+09,-990000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,abcd
1109,0.000000e+00,0.271026,8.144805e+09,0.000000e+00,0.000000e+00,5.439274e+09,6.821990e+08,1.748272e+10,7.462606e+09,-1056000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,abcd
1110,0.000000e+00,0.277900,5.070331e+09,0.000000e+00,0.000000e+00,3.206802e+09,6.223360e+08,9.886805e+09,4.447995e+09,-1007000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,abcd


In [ ]:
student_details["symbol"]